<a href="https://colab.research.google.com/github/WilliamShengYangHuang/RC18_GenAI/blob/main/Video_Frame_Sampler_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Necessary Packages

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Extract Frames

In [ ]:
# Set the input video path (replace 'your_video.mp4' with the actual file path)
video_path = '/content/drive/MyDrive/Landscape/StyleGAN2_Interpolation.mp4'

# Create the output folder path in Google Drive
output_folder = '/content/drive/MyDrive/landscape_frame_captures' #@param{type:'string'}

# Check if the folder exists; if not, create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Folder created: {output_folder}")
else:
    print(f"Folder already exists: {output_folder}")

# Define the number of frames to extract per second
frames_per_second = 2  #@param{type:'slider', min:1, max:30, step:1}

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get the video frame rate (frames per second)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_interval = int(fps / frames_per_second) if fps > 0 else 1  # Calculate the interval between frames to save

# Initialise frame counters
frame_count = 0
saved_frame_count = 0

# Loop through the video frames
while cap.isOpened():
    ret, frame = cap.read()  # Read the next frame
    if not ret:
        break  # Exit the loop if no frame is read (end of video)

    # Check if the current frame is one to save
    if frame_count % frame_interval == 0:
        output_path = os.path.join(output_folder, f'frame_{saved_frame_count:04d}.png')  # Set the output file name
        cv2.imwrite(output_path, frame)  # Save the frame as an image file
        saved_frame_count += 1  # Increment the count of saved frames

    frame_count += 1  # Increment the total frame count

# Release the video capture object
cap.release()

# Print the total number of frames saved
print(f'Saved {saved_frame_count} images to the folder "{output_folder}"')



## Plot Frame Grid

In [ ]:
# Load all image files
image_files = sorted([os.path.join(output_folder, img) for img in os.listdir(output_folder) if img.endswith('.png')])

# Set the size of the image grid
rows= 8 #@param{type:'integer'}
cols = 15 #@param{type:'integer'}
num_images = rows * cols

# Ensure there are enough images to fill the grid
if len(image_files) < num_images:
    raise ValueError(f"At least {num_images} images are required to form a {rows}x{cols} grid, but only {len(image_files)} images were found.")

# Read the first rows * cols images and resize them
images = [cv2.imread(img) for img in image_files[:num_images]]
img_height, img_width, _ = images[0].shape

# Create a blank canvas with dimensions (rows * img_height) x (cols * img_width)
canvas = Image.new('RGB', (cols * img_width, rows * img_height))

# Paste each image onto the canvas
for i in range(rows):
    for j in range(cols):
        img = Image.fromarray(cv2.cvtColor(images[i * cols + j], cv2.COLOR_BGR2RGB))
        canvas.paste(img, (j * img_width, i * img_height))

# Set DPI and save the resulting image
output_image_path = '/content/drive/MyDrive/frame_matrix_output_3.png' #@param{type:'string'}
canvas.save(output_image_path, dpi=(200, 200))
print(f'The image grid has been saved to {output_image_path}')

# Display the image grid
plt.figure(figsize=(15, 8))
plt.imshow(canvas)
plt.axis('off')
plt.show()
